<a href="https://colab.research.google.com/github/Guts-war/Alura-imers-o-ia-2025/blob/main/IAtube_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q google-adk

import os
from google.colab import userdata
from google import genai
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types
from datetime import date
import textwrap
from IPython.display import display, Markdown
import warnings

warnings.filterwarnings("ignore")

# Configura a API Key do Google Gemini
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
client = genai.Client()
model = "gemini-2.0-flash"

# Função auxiliar para enviar mensagem ao agente e obter resposta
def call_agent(agent: Agent, message_text: str) -> str:
    session_service = InMemorySessionService()
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    content = types.Content(role="user", parts=[types.Part(text=message_text)])
    final_response = ""
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
            for part in event.content.parts:
                if part.text is not None:
                    final_response += part.text + "\n"
    return final_response

# Função auxiliar para exibir texto formatado
def to_markdown(text):
    text = text.replace('•', '  *')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

# Agente 1: Buscador de Conteúdo
def agente_buscador(topico, data_de_hoje):
    buscador = Agent(
        name="agente_buscador",
        model=model,
        description="Busca conteúdos relevantes na web.",
        tools=[google_search],
        instruction="""
        Você é um agente de pesquisa na internet. Sua tarefa é usar a ferramenta de busca do Google
        (google_search) para encontrar os lançamentos e notícias mais recentes e relevantes sobre
        o tópico fornecido. Foque em conteúdos que sejam atuais, no máximo um mês antes da data de hoje.
        """
    )
    entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"
    conteudos_buscados = call_agent(buscador, entrada_do_agente_buscador)
    return conteudos_buscados

# Agente 2: Selecionador de Conteúdo
def agente_selecionador(topico, conteudos_buscados):
    selecionador = Agent(
        name="agente_selecionador",
        model=model,
        description="Seleciona os conteúdos mais relevantes para o vídeo.",
        instruction="""
        Você é um especialista em conteúdo para YouTube. A partir dos conteúdos encontrados pelo
        agente de busca, você deve selecionar os mais relevantes e interessantes para um vídeo no YouTube
        sobre o tópico fornecido. Considere a relevância, o potencial de engajamento e a adequação ao público-alvo.
        Apresente os conteúdos selecionados em formato de lista, destacando os principais pontos de cada um.
        """,
    )
    entrada_do_agente_selecionador = f"Tópico: {topico}\nConteúdos buscados: {conteudos_buscados}"
    conteudos_selecionados = call_agent(selecionador, entrada_do_agente_selecionador)
    return conteudos_selecionados

# Agente 3: Criador de Roteiro
def agente_criador_roteiro(topico, conteudos_selecionados):
    criador_roteiro = Agent(
        name="agente_criador_roteiro",
        model=model,
        description="Cria um roteiro de vídeo para o YouTube.",
        instruction="""
        Você é um roteirista de vídeos para o YouTube. Com base nos conteúdos selecionados,
        crie um roteiro completo e detalhado para um vídeo no YouTube sobre o tópico fornecido.
        O roteiro deve incluir:
        - Título do vídeo
        - Introdução (gancho, apresentação do tema)
        - Desenvolvimento (divisão em seções, detalhes dos conteúdos, exemplos)
        - Ideias de elementos visuais (imagens, vídeos, gráficos) para cada seção
        - Sugestões de música de fundo e efeitos sonoros
        - Chamada para ação (CTA)
        - Encerramento
        Use uma linguagem clara, concisa e envolvente.
        """,
    )
    entrada_do_agente_criador_roteiro = f"Tópico: {topico}\nConteúdos selecionados: {conteudos_selecionados}"
    roteiro_do_video = call_agent(criador_roteiro, entrada_do_agente_criador_roteiro)
    return roteiro_do_video

# Agente 4: Produtor Multimídia
def agente_produtor_multimidia(topico, roteiro_do_video):
    produtor_multimidia = Agent(
        name="agente_produtor_multimidia",
        model=model,
        description="Gera os arquivos de multimídia para o vídeo.",
        instruction="""
        Você é um produtor de multimídia. A partir do roteiro do vídeo, você deve gerar os arquivos
        necessários para a produção do vídeo, incluindo:
        - Imagens (descreva as imagens necessárias para cada seção do vídeo)
        - Vídeos (descreva os vídeos necessários, se houver)
        - Áudios (descreva a música de fundo e os efeitos sonoros necessários)
        Especifique os tipos de arquivos, formatos e durações aproximadas.
        E informe links, de onde o conteúdo pode ser baixado.
        """,
    )
    entrada_do_agente_produtor_multimidia = f"Tópico: {topico}\nRoteiro do vídeo: {roteiro_do_video}"
    arquivos_multimidia = call_agent(produtor_multimidia, entrada_do_agente_produtor_multimidia)
    return arquivos_multimidia

# Função principal para executar o chatbot
def criar_video_youtube():
    data_de_hoje = date.today().strftime("%d/%m/%Y")
    print("🚀 Iniciando o Sistema de Criação de Vídeos para o YouTube com 4 Agentes 🚀")
    topico = input("❓ Por favor, digite o TÓPICO sobre o qual você quer criar o vídeo: ")

    if not topico:
        print("❌ Tópico inválido. Por favor, insira um tópico válido.")
        return

    print(f"Maravilha, vamos criar um vídeo sobre {topico}!")

    conteudos_buscados = agente_buscador(topico, data_de_hoje)
    print("\n--- Resultado do Agente 1 (Buscador de Conteúdo) ---\n")
    display(to_markdown(conteudos_buscados))
    print("---------------------------------\n")

    conteudos_selecionados = agente_selecionador(topico, conteudos_buscados)
    print("\n--- Resultado do Agente 2 (Selecionador de Conteúdo) ---\n")
    display(to_markdown(conteudos_selecionados))
    print("---------------------------------\n")

    roteiro_do_video = agente_criador_roteiro(topico, conteudos_selecionados)
    print("\n--- Resultado do Agente 3 (Criador de Roteiro) ---\n")
    display(to_markdown(roteiro_do_video))
    print("---------------------------------\n")

    arquivos_multimidia = agente_produtor_multimidia(topico, roteiro_do_video)
    print("\n--- Resultado do Agente 4 (Produtor Multimídia) ---\n")
    display(to_markdown(arquivos_multimidia))
    print("---------------------------------\n")

if __name__ == "__main__":
    criar_video_youtube()